In [1]:
"""
This file demonstrates a very simple agent with:
1. **Short-term memory** (cleared every run)
2. **Long-term memory** (stored on disk in JSON)
3. A CrewAI Agent that uses long-term memory to enrich explanations
4. A structured and robust prompt that avoids hallucinations

This example is beginner-friendly and intentionally transparent.
"""

import os, json, pathlib, time
from datetime import datetime
from typing import Dict, Any, List
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv
from datetime import datetime
import os

load_dotenv() 
# ---------------------------------------------------------------------
# 1. LOAD THE LANGUAGE MODEL
# ---------------------------------------------------------------------
# We wrap a LangChain model using CrewAI's LLM abstraction.
# If OpenAI or other provider isn't available, fallback to None.
try:
    from crewai import LLM
    llm = LLM(model=os.getenv("MODEL_NAME", "gpt-4o-mini"))
except Exception:
    llm = None   # The Agent will fail gracefully instead of crashing


# ---------------------------------------------------------------------
# 2. LONG-TERM MEMORY (FILE-BASED)
# ---------------------------------------------------------------------
# This section creates a small folder containing:
#   cb_memory_simple/longterm.json
#
# The long-term memory persists **across runs**, unlike short-term memory.
# We store simple dictionary objects like:
#   {
#       "analogy_library": ["lock & key", "signal beacons"],
#       "encryption_fact": {...}
#   }
#
# In real Agentic AI systems, long-term memory can be:
#   - vector databases
#   - semantic search stores
#   - relational databases
#   - timeline stores
#   - graph memories
BASE = pathlib.Path("./cb_memory_simple")
BASE.mkdir(exist_ok=True)
LT = BASE / "longterm.json"

def _load(p: pathlib.Path, default):
    """Load JSON safely from disk. If file missing or invalid → return default."""
    if not p.exists(): 
        return default
    try:
        return json.loads(p.read_text())
    except Exception:
        return default

def _save(p: pathlib.Path, data: Any):
    """Save JSON safely to disk with pretty formatting."""
    p.write_text(json.dumps(data, ensure_ascii=False, indent=2))


class Memory:
    """
    A tiny long-term memory manager.
    It loads memory on startup and saves every time you write.
    """
    def __init__(self):
        self.longterm = _load(LT, {})   # dict of {key: {"value": ..., "updated_at": ...}}

    def recall_longterm(self, keys: List[str]) -> Dict[str, Any]:
        """Return only the keys requested. Missing ones are skipped."""
        return {k: self.longterm.get(k) for k in keys if k in self.longterm}

    def write_longterm(self, key: str, value: Any):
        """Write/update long-term memory with timestamp."""
        self.longterm[key] = {
            "value": value,
            "updated_at": datetime.utcnow().isoformat()
        }
        _save(LT, self.longterm)


# Initialize persistent memory
mem = Memory()

# Seed an analogy library if none exists
if "analogy_library" not in mem.longterm:
    mem.write_longterm("analogy_library", ["lock & key", "secret courier", "signal beacons"])



# ---------------------------------------------------------------------
# 3. SHORT-TERM MEMORY (RESET EVERY RUN)
# ---------------------------------------------------------------------
# This is the classic "working memory" for the current conversation.
# It is *not* saved.
short_term_memory = {}



# ---------------------------------------------------------------------
# 4. PERSONA + AGENT DEFINITION
# ---------------------------------------------------------------------
SYSTEM = (
    "You are Captain Byte, a cyber adventure guide who turns technical ideas "
    "into short, vivid mini-stories WITHOUT breaking factual accuracy. "
    "You are imaginative but remain grounded in verified facts. "
    "Avoid adding speculative details not provided in the context."
)



agent = Agent(
    role="Cyber Adventure Guide",
    goal="Explain a technical topic as a brief, vivid adventure that remains scientifically accurate.",
    backstory="Captain Byte sails the Digital Seas helping rookies understand the invisible forces of technology.",
    llm=llm,
    allow_delegation=False,   # for beginners: avoid unexpected multi-agent complexity
    verbose=True,
)



# ---------------------------------------------------------------------
# 5. TOOL: Micro factual lookup (simulated)
# ---------------------------------------------------------------------
def lore_lookup(topic: str) -> str:
    """Returns a tiny factual snippet about the topic (no hallucinations)."""
    data = {
        "encryption": "Encryption turns readable plaintext into scrambled ciphertext using keys.",
        "black holes": "Black holes have an event horizon where gravity is so strong that not even light escapes.",
    }
    return data.get(topic.lower(), f"No verified nugget for '{topic}'")



# ---------------------------------------------------------------------
# 6. MAIN EXECUTION LOOP
# ---------------------------------------------------------------------
def run_once(topic: str) -> Dict[str, Any]:
    """
    A single full run:
      - Load short-term memory
      - Retrieve long-term analogies
      - Build structured prompt
      - Run the CrewAI agent
      - Optionally write new facts to long-term memory
    """

    # ==== STEP 1 — Reset short-term memory for this run ====
    short_term_memory.clear()
    short_term_memory["topic"] = topic
    short_term_memory["nugget"] = lore_lookup(topic)

    # Retrieve analogy library from long-term memory
    short_term_memory["analogy_library"] = (
        mem.recall_longterm(["analogy_library"])
        .get("analogy_library", {})
        .get("value", [])
    )

    # ==== STEP 2 — Compose transparent context ====
    context = (
        f"# CONTEXT\n"
        f"- Topic: {short_term_memory['topic']}\n"
        f"- Verified Nugget: {short_term_memory['nugget']}\n"
        f"- Analogy Options: {short_term_memory['analogy_library']}\n"
    )

    # ==== STEP 3 — Build a robust user prompt ====
    user_prompt = f"""{SYSTEM}

                    {context}

                    # TASK
                    Explain the topic **"{topic}"** as a short adventure story for a beginner.
                    
                    ## RULES
                    - Use **two short paragraphs**.
                    - Limit total length to **≤150 words**.
                    - ONLY use facts you are **confident** about.
                    - If information is limited, keep the story simple rather than inventing details.
                    - Prefer using ONE analogy from the list if relevant.
                    - Maintain a friendly and imaginative tone without compromising accuracy.
                    
                    Begin your explanation below:
                    """

    # ==== STEP 4 — Define the CrewAI Task ====
    task = Task(
        description=user_prompt,
        expected_output="Two short paragraphs no longer than 150 words; vivid and factually correct.",
        agent=agent,
    )

    crew = Crew(
        agents=[agent],
        tasks=[task],
        process=Process.sequential,
        verbose=True
    )

    # ==== STEP 5 — Run the agent ====
    out = crew.kickoff()
    text = str(out)

    # ==== STEP 6 — Promote verified nuggets to long-term memory ====
    if (
        f"{topic}_fact" not in mem.longterm
        and "No verified nugget" not in short_term_memory["nugget"]
    ):
        mem.write_longterm(f"{topic}_fact", short_term_memory["nugget"])

    return {"topic": topic, "text": text}



# ---------------------------------------------------------------------
# 7. MAIN PROGRAM
# ---------------------------------------------------------------------
if __name__ == "__main__":
    t0 = time.time()
    topic = os.getenv("TOPIC", "Biophysics")

    print(f"\n=== Captain Byte (Short + Long Memory) on '{topic}' ===\n")

    result = run_once(topic)
    dt = time.time() - t0

    print("\n--- Output ---\n")
    print(result["text"])

    print("\n--- Stats ---")
    print({"elapsed_s": round(dt, 2)})

    print("\nLong-term memory file:", str(LT))



=== Captain Byte (Short + Long Memory) on 'Biophysics' ===



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: bb6df663-b810-4ba3-ac4b-bf40e6250499                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Cyber Adventure Guide                                                                                   │
│                                                                                                                 │
│  Task: You are Captain Byte, a cyber adventure guide who turns technical ideas into short, vivid mini-stories   │
│  WITHOUT breaking factual accuracy. You are imaginative but remain grounded in verified facts. Avoid adding     │
│  speculative details not provided in the context.                                                               │
│                                                                                                                 │
│                      # CONTEXT                                                                                  │
│  - Topic: Biophysics                                                                                            │
│  - Verified Nugget: No verified nugget for 'Biophysics'                                                         │
│  - Analogy Options: ['lock & key', 'secret courier', 'signal beacons']                                          │
│                                                                                                                 │
│                                                                                                                 │
│                      # TASK                                                                                     │
│                      Explain the topic **"Biophysics"** as a short adventure story for a beginner.              │
│                                                                                                                 │
│                      ## RULES                                                                                   │
│                      - Use **two short paragraphs**.                                                            │
│                      - Limit total length to **≤150 words**.                                                    │
│                      - ONLY use facts you are **confident** about.                                              │
│                      - If information is limited, keep the story simple rather than inventing details.          │
│                      - Prefer using ONE analogy from the list if relevant.                                      │
│                      - Maintain a friendly and imaginative tone without compromising accuracy.                  │
│                                                                                                                 │
│                      Begin your explanation below:                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Cyber Adventure Guide                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  In the grand realm of biophysics, imagine a secret courier navigating through a bustling city. This courier    │
│  represents the molecules in our bodies, delivering critical messages and resources. Just like the courier      │
│  uses maps and navigational tools, biophysics employs the principles of physics to understand how these         │
│  molecules move, interact, and perform their vital functions. The interplay of forces and energies at the       │
│  molecular level is what keeps life thriving.                                                                   │
│                                                                                                                 │
│  As the courier delivers parcels, they make decisions based on the "terrain" of biological environments like    │
│  cells and tissues. Biophysics reveals the hidden pathways and obstacles encountered, ensuring timely and       │
│  precise deliveries. This field unravels the mysteries of life through the lens of physics, helping us          │
│  understand everything from muscle movement to how our DNA replicates. Join us on this thrilling journey,       │
│  where the science of life dances in harmony with the laws of nature!                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bee65637-0e06-48fc-adda-23c0df20d6af                                                                     │
│  Agent: Cyber Adventure Guide                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: bb6df663-b810-4ba3-ac4b-bf40e6250499                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: In the grand realm of biophysics, imagine a secret courier navigating through a bustling city.   │
│  This courier represents the molecules in our bodies, delivering critical messages and resources. Just like     │
│  the courier uses maps and navigational tools, biophysics employs the principles of physics to understand how   │
│  these molecules move, interact, and perform their vital functions. The interplay of forces and energies at     │
│  the molecular level is what keeps life thriving.                                                               │
│                                                                                                                 │
│  As the courier delivers parcels, they make decisions based on the "terrain" of biological environments like    │
│  cells and tissues. Biophysics reveals the hidden pathways and obstacles encountered, ensuring timely and       │
│  precise deliveries. This field unravels the mysteries of life through the lens of physics, helping us          │
│  understand everything from muscle movement to how our DNA replicates. Join us on this thrilling journey,       │
│  where the science of life dances in harmony with the laws of nature!                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Output ---

In the grand realm of biophysics, imagine a secret courier navigating through a bustling city. This courier represents the molecules in our bodies, delivering critical messages and resources. Just like the courier uses maps and navigational tools, biophysics employs the principles of physics to understand how these molecules move, interact, and perform their vital functions. The interplay of forces and energies at the molecular level is what keeps life thriving.

As the courier delivers parcels, they make decisions based on the "terrain" of biological environments like cells and tissues. Biophysics reveals the hidden pathways and obstacles encountered, ensuring timely and precise deliveries. This field unravels the mysteries of life through the lens of physics, helping us understand everything from muscle movement to how our DNA replicates. Join us on this thrilling journey, where the science of life dances in harmony with the laws of nature!

--- Stats ---
{'elapsed_s